In [4]:
import numpy as np
import pandas as pd
import os

def load_data(folder_landmarks, folder_motion, folder_velocity, class_label):
    X = []
    y = []
    filenames = os.listdir(folder_landmarks)
    for fname in filenames:
        if not fname.endswith('_landmarks.npy'):
            continue

        # Match filenames across all 3 folders
        landmark_file = os.path.join(folder_landmarks, fname)
        motion_file = os.path.join(folder_motion, fname.replace('_landmarks.npy', '_motion.npy'))
        velocity_file = os.path.join(folder_velocity, fname.replace('_landmarks.npy', '_velocity.npy'))

        if not os.path.exists(motion_file) or not os.path.exists(velocity_file):
            continue

        landmarks = np.load(landmark_file).flatten()
        motion = np.load(motion_file).flatten()
        velocity = np.load(velocity_file).flatten()

        features = np.concatenate([landmarks, motion, velocity])
        X.append(features)
        y.append(class_label)
    return X, y

# Load and combine training data
X_all, y_all = [], []
for i, cls in enumerate(['Non-suspicious', 'Suspicious']):
    Xc, yc = load_data(
        f'transformed/landmarks.npy/Train/{cls}',
        f'transformed/motion.npy/Train/{cls}',
        f'transformed/velocity.npy/Train/{cls}',
        i  # 0 = Non-suspicious, 1 = Suspicious
    )
    X_all.extend(Xc)
    y_all.extend(yc)

# Convert to DataFrame
df = pd.DataFrame(X_all)
df['label'] = y_all  # Add label column

# Save to CSV
os.makedirs("csv_output", exist_ok=True)
csv_path = "csv_output/landmarks_motion_velocity_dataset.csv"
df.to_csv(csv_path, index=False)
print(f"Saved dataset to {csv_path}")


Saved dataset to csv_output/landmarks_motion_velocity_dataset.csv


In [ ]:
import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load features from .npy files and flatten them
def load_data(folder_landmarks, folder_motion, folder_velocity, class_label):
    X = []
    y = []
    for fname in os.listdir(folder_landmarks):
        if not fname.endswith('_landmarks.npy'):
            continue

        base = fname.replace('_landmarks.npy', '')
        path_lm = os.path.join(folder_landmarks, fname)
        path_mo = os.path.join(folder_motion, base + '_motion.npy')
        path_vel = os.path.join(folder_velocity, base + '_velocity.npy')

        if not os.path.exists(path_mo) or not os.path.exists(path_vel):
            continue

        # Load and flatten all arrays
        landmarks = np.load(path_lm).flatten()
        motion = np.load(path_mo).flatten()
        velocity = np.load(path_vel).flatten()

        features = np.concatenate([landmarks, motion, velocity])
        X.append(features)
        y.append(class_label)
    return X, y

# Load training data
X_train, y_train = [], []
for i, cls in enumerate(['Non-suspicious', 'Suspicious']):
    Xc, yc = load_data(
        # f'transformed/landmarks.npy/Train/{cls}',
        f'transformed/motion.npy/Train/{cls}',
        f'transformed/velocity.npy/Train/{cls}',
        i
    )
    X_train.extend(Xc)
    y_train.extend(yc)

# Load test data
X_test, y_test = [], []
for i, cls in enumerate(['Non-suspicious', 'Suspicious']):
    Xc, yc = load_data(
        # f'transformed/landmarks.npy/Test/{cls}',
        f'transformed/motion.npy/Test/{cls}',
        f'transformed/velocity.npy/Test/{cls}',
        i
    )
    X_test.extend(Xc)
    y_test.extend(yc)

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Decision Tree
clf = DecisionTreeClassifier(max_depth=10, random_state=42)
clf.fit(X_train_scaled, y_train)

# Evaluate
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=['Non-suspicious', 'Suspicious']))


                precision    recall  f1-score   support

Non-suspicious       0.61      0.61      0.61        18
    Suspicious       0.61      0.61      0.61        18

      accuracy                           0.61        36
     macro avg       0.61      0.61      0.61        36
  weighted avg       0.61      0.61      0.61        36




KeyboardInterrupt



KeyboardInterrupt: 